# Exercise 04:  Route Planning

In route planning, the objective is to find the best way to get from point A to point B (think Google Maps). In this exercise, we will try top of the classic shortest path problem.

### Basic Imports

In [1]:
from typing import List, Tuple
import os
import json
from typing import List, Optional
import graderUtil
import util
from mapUtil import (
    CityMap,
    checkValid,
    createGridMap,
    createGridMapWithCustomTags,
    createStanfordMap,
    getTotalCost,
    locationFromTag,
    makeGridLabel,
    makeTag,
)

from util import Heuristic, SearchProblem, State, UniformCostSearch

### Basic Functions

In [10]:
def extractPath(startLocation: str, search: util.SearchAlgorithm) -> List[str]:
    """
    Assumes that `solve()` has already been called on the `searchAlgorithm`.

    We extract a sequence of locations from `search.path` (see util.py to better
    understand exactly how this list gets populated).
    """
    return [startLocation] + search.actions

def printPath(
    path: List[str],
    waypointTags: List[str],
    cityMap: CityMap,
    outPath: Optional[str] = "path.json",
):
    doneWaypointTags = set()
    for location in path:
        for tag in cityMap.tags[location]:
            if tag in waypointTags:
                doneWaypointTags.add(tag)
        tagsStr = " ".join(cityMap.tags[location])
        doneTagsStr = " ".join(sorted(doneWaypointTags))
        print(f"Location {location} tags:[{tagsStr}]; done:[{doneTagsStr}]")
    print(f"Total distance: {getTotalCost(path, cityMap)}")

    # (Optional) Write path to file, for use with `visualize.py`
    if outPath is not None:
        with open(outPath, "w") as f:
            data = {"waypointTags": waypointTags, "path": path}
            json.dump(data, f, indent=2)

### Task 1: Shortest Path Problem

Implement ``ShortestPathProblem`` so that given a ``startLocation`` and ``endTag``, the minimum cost path corresponds to the shortest path from startLocation to any location that has the ``endTag``.

In particular, you need to implement ``startState()``, ``isEnd(state)``, ``successorsAndCosts(state)``.

Recall the separation between search problem (modeling) and search algorithm (inference). You should focus on modeling (defining the ``ShortestPathProblem``); the default search algorithm, ``UniformCostSearch`` (UCS), is implemented for you in ``util.py``.


---

In [4]:
class ShortestPathProblem(SearchProblem):
    """
    Defines a search problem that corresponds to finding the shortest path
    from `startLocation` to any location with the specified `endTag`.
    """

    def __init__(self, startLocation: str, endTag: str, cityMap: CityMap):
        self.startLocation = startLocation
        self.endTag = endTag
        self.cityMap = cityMap

    def startState(self) -> State:
        # The start state is the start location with an empty tuple as memory
        return State(self.startLocation, memory=())

    def isEnd(self, state: State) -> bool:
        # Check if the current location matches the endTag
        return self.endTag in self.cityMap.tags[state.location]

    def successorsAndCosts(self, state: State) -> List[Tuple[str, State, float]]:
        successors = []
        for neighbor in self.cityMap.distances[state.location]:
            cost = self.cityMap.distances[state.location][neighbor]
            # Using an empty tuple for memory in the successor states
            successors.append((neighbor, State(neighbor, memory=()), cost))
        return successors

---

Let us find the shortest path from gate to the landmark of Stanford university, Hoover Tower.

### Initialization Setting

In [18]:
cityMap = createStanfordMap()
startLocation = locationFromTag(makeTag("landmark", "gates"), cityMap)
endTag = makeTag("landmark", "hoover_tower")

### Print Path

In [19]:
ucs = util.UniformCostSearch(verbose=0)
ucs.solve(ShortestPathProblem(startLocation,endTag,cityMap))
path = extractPath(startLocation, ucs)
printPath(path=path, waypointTags=[], cityMap=cityMap)

Location 5060846126 tags:[label=5060846126 landmark=gates]; done:[]
Location 6333452711 tags:[label=6333452711 crossing=unmarked highway=crossing]; done:[]
Location 256568547 tags:[label=256568547]; done:[]
Location 5444520501 tags:[label=5444520501]; done:[]
Location 5444520500 tags:[label=5444520500]; done:[]
Location 5444520499 tags:[label=5444520499]; done:[]
Location 256568551 tags:[label=256568551]; done:[]
Location 256568564 tags:[label=256568564]; done:[]
Location 5444520498 tags:[label=5444520498]; done:[]
Location 5444520497 tags:[label=5444520497]; done:[]
Location 5444520496 tags:[label=5444520496]; done:[]
Location 256568553 tags:[label=256568553]; done:[]
Location 256568550 tags:[label=256568550]; done:[]
Location 6333452712 tags:[label=6333452712 crossing=unmarked highway=crossing]; done:[]
Location 6883032631 tags:[label=6883032631 bus=yes name=Packard Electrical Engineering network=Marguerite operator=Stanford University public_transport=stop_position]; done:[]
Locatio

### Visualization

In [15]:
os.system("python visualization.py")

0